In [ ]:
#|default_exp prompt_chain


From the [Anthropic Effective Agents Guide](https://www.anthropic.com/research/building-effective-agents)
![](https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F7418719e3dab222dccb379b8879e1dc08ad34c78-2401x1000.png&w=3840&q=75)
Prompt chaining decomposes a task into a sequence of steps, where each LLM call processes the output of the previous one. You can add programmatic checks (see "gate” in the diagram below) on any intermediate steps to ensure that the process is still on track.

**When to use this workflow:**This workflow is ideal for situations where the task can be easily and cleanly decomposed into fixed subtasks. The main goal is to trade off latency for higher accuracy, by making each LLM call an easier task.

Examples where prompt chaining is useful:

- Generating Marketing copy, then translating it into a different language.
- Writing an outline of a document, checking that the outline meets certain criteria, then writing the document based on the outline.

In [ ]:
#| default_exp chain


In [ ]:
#| export
from cosma.core import Agent
from dataclasses import dataclass, field
from typing import List, Callable, Dict, Any, Optional
from fastcore.basics import patch
from cosette import models


In [ ]:
from IPython.display import display, Markdown

In [ ]:
#| export
@dataclass
class Chain:
    """A sequence of agents that process information with flexible input/output types and validation"""
    name: str
    agents: List[Agent] = field(default_factory=list)
    steps: List[dict] = field(default_factory=list)
    input_type: str = "text"  # text, image, mixed, structured
    output_type: str = "text"
    validators: List[Callable] = field(default_factory=list)
    
    def __post_init__(self):
        """Initialize chain with type checking and validation setup"""
        self.history = []
        
    @property
    def use(self):
        """Track token usage across chain"""
        return sum((a.chat.use for a in self.agents), start=0)
    
    def validate(self, step_output, step_num):
        """Run validators appropriate for the output type"""
        results = []
        for v in self.validators:
            results.append(v(step_output))
        return all(results)

In [ ]:
#| export
@patch
def __call__(self:Chain, x):
    """Execute the chain on input x"""
    result = x
    for a in self.agents:
        curr_input = result
        result = a.run_with_tools(result)
        self.steps.append(dict(agent=a, input=curr_input, output=result))
    return result

In [ ]:
#| export
@patch
def show(self:Chain):
    """Display chain execution with clear, readable formatting"""
    md = [f"# {self.name}\n"]
    
    # Show chain structure
    md.append("## Structure")
    for i,a in enumerate(self.agents): 
        md.append(f"{i+1}. {a.role}")
        if i < len(self.agents)-1: md.append("   ↓")
    
    # Show execution steps
    if self.steps:
        md.append("\n## Execution")
        for i,s in enumerate(self.steps):
            md.append(f"\nStep {i+1}: {s['agent'].role}")
            md.append(f"\nInput:\n```\n{s['input']}\n```")
            md.append(f"\nOutput:\n```\n{s['output']}\n```")
            if i < len(self.steps)-1: md.append("\n   ↓")
    
    return "\n".join(md)


In [ ]:
# Let's use Cosette to grab a model 
model = models[2]
model

'gpt-4o'

In [ ]:
# Create our test agents
writer = Agent("writer", model, system="Create concise, engaging product descriptions")
translator = Agent("translator", model, system="Translate while maintaining tone and style")

# Simple chain
chain = Chain("product_translator")
chain.agents = [writer, translator]

# Test it
prompt = "Describe a smart coffee maker"
result = chain(prompt)
display(Markdown(chain.show()))


# product_translator

## Structure
1. writer
   ↓
2. translator

## Execution

Step 1: writer

Input:
```
Describe a smart coffee maker
```

Output:
```
Brew smarter with the Smart Coffee Maker, the ultimate companion for coffee enthusiasts. This innovative machine lets you customize your brew to perfection with precise temperature and strength controls. Easily schedule your morning cup using a user-friendly app, or command it with voice control via Alexa or Google Assistant. Featuring a sleek design, this coffee maker seamlessly integrates into any modern kitchen, offering rapid heating technology for a perfect cup every time. Enjoy exceptional flavor and convenience with every sip.
```

   ↓

Step 2: translator

Input:
```
Brew smarter with the Smart Coffee Maker, the ultimate companion for coffee enthusiasts. This innovative machine lets you customize your brew to perfection with precise temperature and strength controls. Easily schedule your morning cup using a user-friendly app, or command it with voice control via Alexa or Google Assistant. Featuring a sleek design, this coffee maker seamlessly integrates into any modern kitchen, offering rapid heating technology for a perfect cup every time. Enjoy exceptional flavor and convenience with every sip.
```

Output:
```
Elabora con inteligencia con la Cafetera Inteligente, la compañera definitiva para los amantes del café. Esta innovadora máquina te permite personalizar tu café hasta la perfección con controles precisos de temperatura e intensidad. Programa fácilmente tu taza matutina usando una aplicación fácil de usar o comándala con control por voz a través de Alexa o Google Assistant. Con un diseño elegante, esta cafetera se integra perfectamente en cualquier cocina moderna, ofreciendo tecnología de calentamiento rápido para una taza perfecta cada vez. Disfruta de un sabor excepcional y comodidad en cada sorbo.
```